In [1]:
#Import library
import pandas as pd
from pathlib import Path
import datetime
import math
import glob

In [2]:
stations = ["v1","v2","y6","y7"]
raw_weather_type = ['薄曇','晴れ','くもり','少雨','弱い雨','強い雨','激しい雨','猛烈な雨','みぞれ(弱い)','雪 (強い)','雪 (弱い)','みぞれ(強い)']
weather_type = ['light cloud', 'sunny', 'cloudy', 'light rain', 'light rain', 'strong rain', 'heavy rain', 'heavy rain', 'sleet (weak)', 'snow (heavy)' ,'snow (weak)','sleet (strong)']

data_dir = Path("./2023_devday_data/")
start_date = ["20220301","20230101"]
end_date = ["20221231","20230228"]
cloud_dir = data_dir / stations[0] / "cloud_{}0000{}.csv".format(stations[0][0], stations[0][1])
sunny_dir = data_dir / stations[0] / "solar_{}0000{}.csv".format(stations[0][0], stations[0][1])
weather_dir = data_dir / stations[0] / "weather_forecast_{}0000{}.csv".format(stations[0][0], stations[0][1])
e_generator_dir = data_dir /stations[0] /  "realne_report_solar_30_{}0000{}_{}_{}.xlsx".format(stations[0][0], stations[0][1], start_date[0], end_date[1])
e_demand_dir = data_dir /stations[0] /  "realne_report_surplus30p_{}0000{}_{}_{}.xlsx".format(stations[0][0], stations[0][1], start_date[0], end_date[0])
e_demand_dir_2 = data_dir /stations[0] /  "realne_report_surplus30p_{}0000{}_{}_{}.xlsx".format(stations[0][0], stations[0][1], start_date[1], end_date[1])

In [3]:
cloud_data = pd.read_csv(cloud_dir)
sunny_data = pd.read_csv(sunny_dir)
weather_data = pd.read_csv(weather_dir)

def save_xls(dict_df, path):
        writer = pd.ExcelWriter(path)
        for key in dict_df:
            dict_df[key].to_excel(writer, key)
        writer.save()

In [4]:
csv_data = pd.merge(weather_data, sunny_data, how = 'outer', on = ["target_date", "execution_date"])
csv_data = pd.merge(csv_data, cloud_data, how = 'outer', on = ["target_date"])
csv_data = csv_data.drop(columns = ["execution_date_x", "execution_date_y"])
csv_data.head()

,target_date,telop_code,telop_name,solar(W/m2),cloud(%)
0,2022-03-25T23:30:00+09:00,21.0,薄曇,0.0,76.0
1,2022-03-25T23:00:00+09:00,21.0,薄曇,0.0,93.0
2,2022-03-25T22:30:00+09:00,21.0,薄曇,0.0,89.0
3,2022-03-25T22:00:00+09:00,10.0,晴れ,0.0,85.0
4,2022-03-25T21:30:00+09:00,10.0,晴れ,0.0,84.0


In [5]:
def csv_process(dataset):
    result = list()
    # Process execution date
    def target_date(sample):
        date, time = sample.split("T")
        time = time.split(":")
        time = time[0] + ":" + time[1]
        return date, time
    
    for index, data in dataset.iterrows():
        sample = dict()
        
        #Target Date
        date, time = target_date(data["target_date"])
        sample["target_date"] = date
        sample["target_time"] = time
        
        #Features
        
        sample["cloud"] = data["cloud(%)"]
        sample["solar"] = data["solar(W/m2)"]
        sample["telop_code"] = data["telop_name"]
        try:
            sample["telop_name"] = weather_type[raw_weather_type.index(data["telop_name"])]
        except:
            sample["telop_name"] = data["telop_name"]
        result.append(sample)
    return result

csv_data= pd.DataFrame(csv_process(csv_data))
csv_data.head()

,target_date,target_time,cloud,solar,telop_code,telop_name
0,2022-03-25,23:30,76.0,0.0,薄曇,light cloud
1,2022-03-25,23:00,93.0,0.0,薄曇,light cloud
2,2022-03-25,22:30,89.0,0.0,薄曇,light cloud
3,2022-03-25,22:00,85.0,0.0,晴れ,sunny
4,2022-03-25,21:30,84.0,0.0,晴れ,sunny


In [6]:
def target_process(path):
    f = pd.ExcelFile(path)
    result = list()
    for sheet_name in f.sheet_names:
        # Storage temp xlsx file
        df = f.parse(sheet_name = sheet_name)
        df = df.drop(labels = [0,1,2,3], axis = 0)
        df = df.to_excel("temp.xlsx", header = None)

        # Read xlsx with right format
        df_new = pd.read_excel("temp.xlsx", index_col = None)        
        for index, data in df_new.iterrows():
            sample = dict()
            sample["target_date"] = sheet_name
            sample["target_time"] = data["時刻"]
            for solar_panel in data.index:
                if len(str(solar_panel)) > 2:
                    sample[solar_panel] = data[solar_panel]
        
            result.append(sample)
    
    return result
e_generator_data = pd.DataFrame(target_process(e_generator_dir))
e_demand_data_v1 = pd.DataFrame(target_process(e_demand_dir))
e_demand_data_v2 = pd.DataFrame(target_process(e_demand_dir_2))


In [8]:
final_dataset = pd.concat([e_demand_data_v1, e_demand_data_v2])
final_dataset = pd.merge(final_dataset, e_generator_data, how = "outer", on = ["target_date", "target_time"])
final_dataset = pd.merge(final_dataset, csv_data, how = "outer", on = ["target_date", "target_time"])

,target_date,target_time,30101:電力量（Wh）,30101:回生電力量（Wh）,30001（Wh）,30011（Wh）,30021（Wh）,30031（Wh）,cloud,solar,telop_code,telop_name
0,2022-03-01,00:00,50201.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2022-03-01,00:30,52129.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2022-03-01,01:00,53018.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2022-03-01,01:30,37823.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2022-03-01,02:00,3897.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17563,2023-03-01,06:30,NaN,NaN,NaN,NaN,NaN,NaN,84.0,79.26,晴れ,sunny
17564,2023-03-01,18:00,NaN,NaN,NaN,NaN,NaN,NaN,50.0,0.00,くもり,cloudy
17565,2023-03-01,15:00,NaN,NaN,NaN,NaN,NaN,NaN,11.0,5705.12,晴れ,sunny
17566,2023-03-01,06:00,NaN,NaN,NaN,NaN,NaN,NaN,84.0,0.00,薄曇,light cloud


In [ ]:
final_dataset.head()